In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

%env SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
%env TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
!echo $SQLITE_TMPDIR
!echo $TMPDIR
!echo $USERPROFILE

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-env-icelake/lib/python3.10/site-packages' at start of search paths.
env: SQLITE_TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
env: TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles
/rds/projects/g/gokhalkm-optimal/DataforCharles



In [11]:
import pytorch_lightning
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
import sqlite3
import logging
import time
from CPRD.data.database.build_static_db import Static
from CPRD.data.database.build_diagnosis_db import Diagnoses
from CPRD.data.database.build_measurements_and_tests_db import Measurements

torch.manual_seed(1337)
logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {device}.")

# !export TMPDIR=/rds/projects/g/gokhalkm-optimal/DataforCharles/tmp
!pwd


Using device: cuda.
/rds/homes/g/gaddcz/Projects/CPRD/data/database


In [3]:
PATH_TO_DB = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/cprd.db"

## Create/load tables

In [7]:
PATH_TO_STATIC = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/baseline/masterDataOptimalWithIMD_v3.csv"
PATH_TO_DIAGNOSIS = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/baseline/masterDataOptimalWithIMD_v3.csv"
PATH_TO_MEASUREMENTS = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/timeseries/measurement_and_tests/lab_measurements/"
load = True

static = Static(PATH_TO_DB, PATH_TO_STATIC, load=load)
diagnosis = Diagnoses(PATH_TO_DB, PATH_TO_DIAGNOSIS, load=load)
measurements = Measurements(PATH_TO_DB, PATH_TO_MEASUREMENTS, load=load)

## View tables

These are built through a job script

In [ ]:
for table in [static, diagnosis, measurements]:
    print(table)

Static table with 28.12M records.
Diagnosis table with 51.00M records.


In [9]:
# print(measurements.measurement_table_names)

['measurement_25_Hydroxyvitamin_D2_level_92', 'measurement_25_Hydroxyvitamin_D3_level_90', 'measurement_AST___aspartate_transam_SGOT__46', 'measurement_AST_serum_level_47', 'measurement_Albumin___creatinine_ratio_37', 'measurement_Basophil_count_22', 'measurement_Blood_calcium_level_38', 'measurement_Blood_urea_28', 'measurement_Body_mass_index_3', 'measurement_Brain_natriuretic_peptide_level_66', 'measurement_Calcium_adjusted_level_41', 'measurement_Calculated_LDL_cholesterol_level_103', 'measurement_Combined_total_vitamin_D2_and_D3_level_93', 'measurement_Corrected_serum_calcium_level_42', 'measurement_Current_smoker_83', 'measurement_Diastolic_blood_pressure_5', 'measurement_Eosinophil_count_21', 'measurement_Erythrocyte_sedimentation_rate_61', 'measurement_Ex_smoker_84', 'measurement_Free_T4_level_76', 'measurement_GFR_calculated_abbreviated_MDRD_34', 'measurement_Haematocrit___PCV_16', 'measurement_Haematocrit_15', 'measurement_Haemoglobin_A1c_level___IFCC_standardised_6', 'measur

## Check query execution time

In [28]:
import time

measurements.connect()

t = time.time()   #  EXPLAIN QUERY PLAN        diagnosis_table           measurement_table        static_table
measurements.cursor.execute(""" SELECT EVENT, COUNT(EVENT)
                                FROM
                                	diagnosis_table
                                 GROUP BY 
                                     EVENT
                                 ;""")
total_count = 0
for condition, count in measurements.cursor.fetchall():
    print(f"{condition}".replace("_"," ").ljust(40) + f" & ${count:,}$ \\\\".rjust(10))
    total_count += count
print("Total".ljust(40) + f" & {total_count:,}")

eval_time = (time.time() - t)
print(eval_time )

ADDISONS DISEASE                         & $6,691$ \\
ADDISON DISEASE                          & $11,794$ \\
AF                                       & $731,332$ \\
ALCOHOLMISUSE V2                         & $1,125,212$ \\
ALLCANCER NOHAEM NOBCC                   & $1,496,973$ \\
ALLERGICRHINITISCONJ                     & $3,291,165$ \\
ALL DEMENTIA                             & $528,602$ \\
ANXIETY                                  & $3,560,978$ \\
ANY DEAFNESS HEARING LOSS V2             & $2,282,766$ \\
AORTICANEURYSM V2                        & $101,134$ \\
ASTHMA PUSHASTHMA                        & $4,175,115$ \\
ATOPICECZEMA                             & $4,369,082$ \\
AUTISM                                   & $156,860$ \\
BIPOLAR                                  & $108,852$ \\
BRONCHIECTASIS                           & $112,618$ \\
CHRONICFATIGUESYNDROMEMM V2              & $82,799$ \\
CHRONIC LIVER DISEASE ALCOHOL            & $63,405$ \\
CKDSTAGE3TO5                           

In [ ]:

count = 0
for mtable in measurements.measurement_table_names:
    measurements.cursor.execute(f"""  SELECT
                                    	COUNT(*)
                                    FROM
                                    	{mtable}
                                    """)
    # WHERE PRACTICE_PATIENT_ID = 'p20684_2602314020684' 
    a = measurements.cursor.fetchall()[0][0]
    print(mtable.ljust(50) + f"{a}")
    count += a
# print(a[:10])
print(count)

# Collector demo

We can extract unique distinct column values from any table, and use these to chunk the data for processing

* We can add practice level conditions here,
    * e.g. practices that are in the North West region. 

In [4]:
from CPRD.data.dataset.collector import SQLiteDataCollector
import polars as pl
import logging 
logging.basicConfig(level=logging.INFO)

collector = SQLiteDataCollector(PATH_TO_DB)
collector.connect()

practice_ids = collector._extract_distinct(["static_table"], "PRACTICE_ID", conditions=["HEALTH_AUTH = 'North West'"])

print(f"The first ten unique practice IDs: {practice_ids[:10]} from {len(practice_ids)} total")

The first ten unique practice IDs: [21505, 20995, 20996, 21508, 21000, 21514, 21515, 21007, 21008, 21010] from 305 total


## Given these practice IDs, we can chain this query to get the pratice patient ID's in a practice

* We do this as only the static table has all practice_patient_id, patient_id, and practice_id stored and indexed.
    * This means we can very quickly find the keys needed to index on the other tables which only have practice_patient_id indexed.
* We can add patient level conditions here
    * For example, only male, people who died during a period, etc

In [5]:
practice_patient_ids = []
for p_id in practice_ids:
    practice_patient_ids.append(collector._extract_distinct(["static_table"], "PRACTICE_PATIENT_ID", conditions=[f"PRACTICE_ID = '{p_id}' AND SEX = 'M'"]))
                                
print(f"\nThe first five unique practice_patient IDs from practice {practice_ids[0]}:\n\t{practice_patient_ids[0][:5]} from {len(practice_patient_ids[0])} total")
print(f"\nTotal male patients in each practice:\n\t {[len(ppid) for ppid in practice_patient_ids]}")


The first five unique practice_patient IDs from practice 21505:
	['p21505_6499892121505', 'p21505_6721456421505', 'p21505_6499676121505', 'p21505_6499427021505', 'p21505_6499706721505'] from 2978 total

Total male patients in each practice:
	 [2978, 7353, 167, 10075, 3456, 10727, 5948, 10060, 3337, 4355, 4359, 4732, 9742, 9991, 7067, 7739, 5589, 2995, 12730, 15698, 8748, 6097, 4769, 7536, 6467, 10999, 8421, 3352, 4423, 13347, 6035, 8694, 11087, 12412, 7594, 2431, 7837, 7384, 4038, 9033, 7273, 6921, 14112, 7506, 6299, 8669, 4480, 2326, 8275, 10382, 3230, 6515, 5382, 3210, 4623, 11147, 4061, 7983, 4202, 9844, 7802, 9414, 6508, 6570, 17641, 14467, 2733, 5349, 36498, 36236, 4853, 13566, 4878, 6545, 231, 6101, 5236, 7443, 7149, 5396, 5926, 6347, 9721, 19320, 3748, 9388, 7912, 10456, 4048, 12342, 12121, 6510, 12210, 11697, 15807, 7896, 10871, 7275, 2947, 8767, 11560, 9993, 7809, 22261, 7067, 4748, 3451, 6236, 9718, 12605, 1077, 5160, 6008, 5280, 4139, 6202, 16834, 3648, 4768, 68067, 11073, 

### Create a generator which chunks the .db tables by practice_id or practice_patient_id and lazily batch using Polars (Rust)

* We can also choose to pass this method a list of conditions, one for each table.
   * For example, we may only want to collect measurements within some predfined list, period etc

In the first case, we may want to generate by individual level values. 

Here, we then pass in the list we generate over - which in this instance is the list of practice patient ID's belonging to the first chunk. Here we are then yielding one person per call to the generator.

In [20]:
generator = collector._lazy_generate_by_distinct(distinct_values=practice_patient_ids, 
                                                 identifier_column="PRACTICE_PATIENT_ID",)
start = time.time()
for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(generator):
    
    print(f"collector took {time.time() - start} seconds")

    start = time.time()
    batch = collector._collate_lazy_tables(lazy_table_frames_dict).collect()
    print(f"collating events took {time.time() - start} seconds")

    start = time.time()
    if _idx > 5:
        break
    
print(lazy_table_frames_dict)
# print(f"Chunk containing practice ID: {chunk_name}\n\n")
for key in lazy_table_frames_dict.keys():
    print(key)
    display(lazy_table_frames_dict[key].collect().head())


INFO:root:Query: SELECT * FROM static_table WHERE PRACTICE_PATIENT_ID IN ('p21505_6499892121505','p21505_672145642150
INFO:root:Query: SELECT * FROM diagnosis_table WHERE PRACTICE_PATIENT_ID IN ('p21505_6499892121505','p21505_672145642
INFO:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D2_level_92 WHERE PRACTICE_PATIENT_ID IN ('p21505_649989
INFO:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D3_level_90 WHERE PRACTICE_PATIENT_ID IN ('p21505_649989
INFO:root:Query: SELECT * FROM measurement_AST___aspartate_transam_SGOT__46 WHERE PRACTICE_PATIENT_ID IN ('p21505_649
INFO:root:Query: SELECT * FROM measurement_AST_serum_level_47 WHERE PRACTICE_PATIENT_ID IN ('p21505_6499892121505','p
INFO:root:Query: SELECT * FROM measurement_Albumin___creatinine_ratio_37 WHERE PRACTICE_PATIENT_ID IN ('p21505_649989
INFO:root:Query: SELECT * FROM measurement_Basophil_count_22 WHERE PRACTICE_PATIENT_ID IN ('p21505_6499892121505','p2
INFO:root:Query: SELECT * FROM measurement_Blood_calcium

collector took 16.61313271522522 seconds
collating events took 0.08240914344787598 seconds


INFO:root:Query: SELECT * FROM diagnosis_table WHERE PRACTICE_PATIENT_ID IN ('p20995_2648958120995','p20995_264880202
INFO:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D2_level_92 WHERE PRACTICE_PATIENT_ID IN ('p20995_264895
INFO:root:Query: SELECT * FROM measurement_25_Hydroxyvitamin_D3_level_90 WHERE PRACTICE_PATIENT_ID IN ('p20995_264895
INFO:root:Query: SELECT * FROM measurement_AST___aspartate_transam_SGOT__46 WHERE PRACTICE_PATIENT_ID IN ('p20995_264
INFO:root:Query: SELECT * FROM measurement_AST_serum_level_47 WHERE PRACTICE_PATIENT_ID IN ('p20995_2648958120995','p
INFO:root:Query: SELECT * FROM measurement_Albumin___creatinine_ratio_37 WHERE PRACTICE_PATIENT_ID IN ('p20995_264895
INFO:root:Query: SELECT * FROM measurement_Basophil_count_22 WHERE PRACTICE_PATIENT_ID IN ('p20995_2648958120995','p2
INFO:root:Query: SELECT * FROM measurement_Blood_calcium_level_38 WHERE PRACTICE_PATIENT_ID IN ('p20995_2648958120995
INFO:root:Query: SELECT * FROM measurement_Blood_urea_28

collector took 21.815685987472534 seconds


INFO:root:Query: SELECT * FROM static_table WHERE PRACTICE_PATIENT_ID IN ('p20996_2622190820996','p20996_262131522099


...

PRACTICE_PATIENT_ID,EVENT,VALUE,DATE
str,str,f64,str
"""p21515_5954187…","""Urine_albumin_…",2.11,"""2011-07-15"""
"""p21515_5954189…","""Urine_albumin_…",2.33,"""2009-04-16"""
"""p21515_5954189…","""Urine_albumin_…",1.88,"""2010-04-13"""
"""p21515_5954189…","""Urine_albumin_…",2.11,"""2011-09-16"""
"""p21515_5954189…","""Urine_albumin_…",3.39,"""2012-05-16"""


lazy_measurement_Urine_microalbumin_creatinine_ratio_36


PRACTICE_PATIENT_ID,EVENT,VALUE,DATE
str,str,f64,str
"""p21515_5955287…","""Urine_microalb…",1.0,"""2000-12-11"""
"""p21515_5955287…","""Urine_microalb…",2.9,"""2000-12-16"""
"""p21515_5955899…","""Urine_microalb…",1.1,"""2014-06-17"""
"""p21515_5956855…","""Urine_microalb…",15.46,"""2013-04-25"""
"""p21515_5957534…","""Urine_microalb…",null,"""2004-05-26"""


lazy_measurement_eGFR_using_creatinine_CKD_EPI_per_1_73_square_metres_33


PRACTICE_PATIENT_ID,EVENT,VALUE,DATE
str,str,f64,str
"""p21515_5954187…","""eGFR_using_cre…",90.0,"""2019-09-18"""
"""p21515_5954187…","""eGFR_using_cre…",90.0,"""2021-09-03"""
"""p21515_5954188…","""eGFR_using_cre…",11.6,"""2019-01-07"""
"""p21515_5954188…","""eGFR_using_cre…",3.8,"""2020-05-30"""
"""p21515_5954188…","""eGFR_using_cre…",3.0,"""2022-02-03"""


In [15]:
display(lazy_table_frames_dict)

{'lazy_static_table': <polars.LazyFrame object at 0x7F653F62A170>}

Alternatively, we may which to generate groups/batches of individuals for vectorisation. In that case we can pass in a list of lists. We generate over the outer list, and each call yields all the inner list values.

For example, in the below we have an outer list of all practices, and an inner list of patients within the practice. Consequently, each call to generate yields all records of all patients within a practice.

In [ ]:
generator = collector._lazy_generate_by_distinct(practice_patient_ids, ["static_table", "measurement_table", "diagnosis_table"], "PRACTICE_PATIENT_ID")
for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(generator):
    break
    
# print(f"Chunk containing practice ID: {chunk_name}\n\n")
display(lazy_table_frames_dict["lazy_static"].collect().head())
display(lazy_table_frames_dict["lazy_measurement"].collect().head())
display(lazy_table_frames_dict["lazy_diagnosis"].collect().head())


### Test collation of generated lazy frames

In [ ]:
lazy_batch = collector._collate_lazy_tables(**lazy_table_frames_dict)

### View how tables are aligned

In [ ]:
batch = lazy_batch.collect()
display(batch)

first_identifier = list(batch["PRACTICE_PATIENT_ID"].unique())[0]
display(first_identifier)
                        
row = batch.filter(pl.col("PRACTICE_PATIENT_ID") == first_identifier)
display(row)


measur = lazy_table_frames_dict["lazy_measurement"].collect()
row = measur.filter(pl.col("EVENT") == "Diastolic_blood_pressure_5")
display(row)

# Demo wrapper

All of this is wrapped together (without the option to filter - add your own wrapper specific to you application)

In [ ]:
# for _idx, (chunk_name, lazy_table_frames_dict) in enumerate(collector.generator()):
#     lazy_batch = collector._collate_lazy_tables(**lazy_table_frames_dict)
#     break
# print(_idx)
# print(lazy_batch.collect())

In [ ]:
!jupyter nbconvert --to html --no-input buildV2.ipynb